In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from deepface import DeepFace
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


###Used to detect emotion in order to test the stressing
def capture_frame():
    js = Javascript('''
    async function captureFrame() {
        const div = document.createElement('div');
        const video = document.createElement('video');
        const stream = await navigator.mediaDevices.getUserMedia({ video: true });

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        const ctx = canvas.getContext('2d');
        ctx.drawImage(video, 0, 0);

        stream.getTracks().forEach(track => track.stop());
        video.remove();

        const data = canvas.toDataURL('image/jpeg');
        return data;
    }
    captureFrame();
    ''')
    display(js)
    data = eval_js('captureFrame()')
    data = data.split(',')[1]
    return b64decode(data)

def analyze_emotion(img):
    try:
        analysis = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)
        if isinstance(analysis, list):
            analysis = analysis[0]
        return analysis['dominant_emotion'], analysis['emotion']
    except Exception as e:
        print(f"Error in emotion detection: {str(e)}")
        return None, None

def draw_results(img, face_detections, emotion, emotions_dict):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if face_detections and emotion:
        for face in face_detections:
            facial_area = face['facial_area']
            x = facial_area['x']
            y = facial_area['y']
            w = facial_area['w']
            h = facial_area['h']

            cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)


            fig, ax = plt.subplots(figsize=(8, 2))
            emotions = list(emotions_dict.keys())
            values = list(emotions_dict.values())

            ax.barh(emotions, values, color='skyblue')
            ax.set_xlabel('Confidence (%)')
            ax.set_title('Emotion Analysis')


            fig.canvas.draw()
            plot_img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            plot_img = plot_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            plt.close()

            label = f"{emotion.upper()}: {emotions_dict[emotion]:.1f}%"
            cv2.putText(img_rgb, label, (x, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return img_rgb

try:
    while True:
        print("Press 'Stop' to interrupt.")
        frame_data = capture_frame()

        with open('current_frame.jpg', 'wb') as f:
            f.write(frame_data)

        img = cv2.imread('current_frame.jpg')

        face_detections = DeepFace.extract_faces(img, enforce_detection=False)
        dominant_emotion, emotions_dict = analyze_emotion(img)

        if dominant_emotion and emotions_dict:

            result_image = draw_results(img, face_detections, dominant_emotion, emotions_dict)
            plt.figure(figsize=(12, 8))
            plt.imshow(result_image)
            plt.axis('off')
            plt.show()
        else:
            print("No face detected or error in emotion analysis")

        plt.close('all')

except KeyboardInterrupt:
    print("Video feed stopped.")

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from deepface import DeepFace
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
#stress detection
def capture_frame():
    js = Javascript('''
    async function captureFrame() {
        const div = document.createElement('div');
        const video = document.createElement('video');
        const stream = await navigator.mediaDevices.getUserMedia({ video: true });

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        const ctx = canvas.getContext('2d');
        ctx.drawImage(video, 0, 0);

        stream.getTracks().forEach(track => track.stop());
        video.remove();

        const data = canvas.toDataURL('image/jpeg');
        return data;
    }
    captureFrame();
    ''')
    display(js)
    data = eval_js('captureFrame()')
    data = data.split(',')[1]
    return b64decode(data)

def analyze_emotion(img):
    try:
        analysis = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)
        if isinstance(analysis, list):
            analysis = analysis[0]
        return analysis['dominant_emotion'], analysis['emotion']
    except Exception as e:
        print(f"Error in emotion detection: {str(e)}")
        return None, None

def analyze_stress(emotions_dict):
    sorted_emotions = sorted(emotions_dict.items(), key=lambda x: x[1], reverse=True)
    top_3_emotions = sorted_emotions[:3]

    emotion_names = [e[0] for e in top_3_emotions]
    emotion_scores = [e[1] for e in top_3_emotions]

    stress_level = 0
    stress_indicators = []


    if 'angry' in emotion_names[:2] and 'fear' in emotion_names[:2]:
        if emotion_names[2] in ['neutral', 'disgust']:
            stress_level = 3  # High stress if angry and fear are in top 2
            stress_indicators.append("Primary stress pattern detected")

    elif ('angry' in emotion_names[:2] or 'fear' in emotion_names[:2]):
        if 'disgust' in emotion_names[:2]:
            stress_level = 2  # Moderate stress if angry / fear are in top two paired with disgust as the other emotion
            stress_indicators.append("Secondary stress pattern detected")

    total_negative = sum(emotions_dict[emotion] for emotion in ['angry', 'fear', 'disgust'])
    if total_negative > 70:
        stress_level = max(stress_level, 2)
        stress_indicators.append("High negative emotion intensity")

    if emotions_dict['happy'] < 10:
        stress_level = max(stress_level, 1)
        stress_indicators.append("Low happiness detected")

    return {
        'stress_level': stress_level,
        'indicators': stress_indicators,
        'top_emotions': top_3_emotions
    }

def draw_results(img, face_detections, stress_analysis, emotions_dict):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if face_detections:
        for face in face_detections:
            facial_area = face['facial_area']
            x = facial_area['x']
            y = facial_area['y']
            w = facial_area['w']
            h = facial_area['h']

            stress_colors = {
                0: (0, 255, 0),    # Green for no stress
                1: (255, 255, 0),  # Yellow for mild stress
                2: (255, 165, 0),  # Orange for moderate stress
                3: (255, 0, 0)     # Red for high stress
            }

            color = stress_colors[stress_analysis['stress_level']]
            cv2.rectangle(img_rgb, (x, y), (x + w, y + h), color, 2)
            stress_levels = ['No', 'Mild', 'Moderate', 'High']
            stress_text = f"Stress Level: {stress_levels[stress_analysis['stress_level']]}"
            y_offset = 30
            cv2.putText(img_rgb, stress_text, (x, y - y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            fig, ax = plt.subplots(figsize=(8, 3))
            emotions = list(emotions_dict.keys())
            values = list(emotions_dict.values())

            ax.barh(emotions, values, color='skyblue')
            ax.set_xlabel('Confidence (%)')
            ax.set_title('Emotion Analysis')
            plt.close()

    return img_rgb

try:
    while True:
        print("Press 'Stop' to interrupt.")
        frame_data = capture_frame()
        with open('current_frame.jpg', 'wb') as f:
            f.write(frame_data)

        img = cv2.imread('current_frame.jpg')

        face_detections = DeepFace.extract_faces(img, enforce_detection=False)
        dominant_emotion, emotions_dict = analyze_emotion(img)

        if dominant_emotion and emotions_dict:
            stress_analysis = analyze_stress(emotions_dict)
            result_image = draw_results(img, face_detections, stress_analysis, emotions_dict)
            if stress_analysis['indicators']:
                print("\nStress Indicators Detected:")
                for indicator in stress_analysis['indicators']:
                    print(f"- {indicator}")
                print("\nTop 3 Emotions:")
                for emotion, score in stress_analysis['top_emotions']:
                    print(f"- {emotion.capitalize()}: {score:.1f}%")

            plt.figure(figsize=(12, 8))
            plt.imshow(result_image)
            plt.axis('off')
            plt.show()
        else:
            print("No face detected or error in emotion analysis")

        plt.close('all')

except KeyboardInterrupt:
    print("Video feed stopped.")

In [ ]:
import cv2
import numpy as np
from keras import models
from google.colab.patches import cv2_imshow

###OLD!!!!
emotion_model = models.load_model('face_model.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'] #labeluri pt emotii


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


img = cv2.imread('selectie2.jpg')
if img is None:
    print("Imaginea nu a fost încarcata corect!")
    exit()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)  # imagine in grey filter
print("Shape:", gray.shape)


faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
print("Fete detectate:", faces)

if len(faces) == 0:
    print("Nu au fost detectate fete!")

for (x, y, w, h) in faces:
    # rois
    face = gray[y:y + h, x:x + w]
    face = cv2.resize(face, (48, 48))  # redimensiune
    face = face / 255.0
    face = np.reshape(face, (1, 48, 48, 1))

    # Prezice emoția
    emotion_prediction = emotion_model.predict(face)
    emotion_index = np.argmax(emotion_prediction)
    emotion_text = emotion_labels[emotion_index]

    # dreptunghi
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Dreptunghi rosu
    cv2.putText(img, emotion_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()
